<a href="https://colab.research.google.com/github/AdrianoHKim/Soulcode/blob/main/notebooks/notebook_convert_parquet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Projeto Final**
Escola: SoulCode Academy

Curso: Bootcamp Analista de Dados - Martech - AD2

**Equipe 08**

**Integrantes**: Wesley, Marcos Paulo, Adriano Kim, Paulo Vitorino, Renato e José Aurelio

Professores: Douglas Ribeiro, Franciane Rodrigues e Jonathas Carneiro

## Preparação de Ambiente

In [ ]:
# Instalaçao Bibliotecas
!pip install gcsfs -q

In [ ]:
# Importando Bibliotecas
import os
import pandas as pd
import random

from google.cloud import storage
from google.colab import data_table
from google.colab import drive
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [ ]:
# Ignorando alguns alertas desnecessários
import warnings
warnings.filterwarnings("ignore")

### Compartilhamento Chave GDrive

In [ ]:
# Cria compartilhamento com Google Drive
drive.mount('/content/drive', force_remount=True)

# Arquivo a ser acessado na pasta compartilhada

target = 'projeto-final-ad2-e8-ae566c3a2c2b.json'

# Caminho completo da pasta compartilhada
folder = '/content/drive/MyDrive/Classroom/AD2 - Analista de Dados/ProjetoFinal'

# Acesso ao arquivo no colab
serviceAccount = os.path.join(folder, target)

Mounted at /content/drive


### Configuração da bucket

In [ ]:
# Nome da bucket
bucket_name = 'projeto-final-ad2-e8'

# Delimitação de pasta
prefix_name = 'dados/tratados/'

In [ ]:
# Conexão com a conta do Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# Conexão com a bucket do Google Cloud
client = storage.Client()
bucket = client.get_bucket(bucket_name)

### Conjuntos de dados

In [ ]:
# Habilita formatação tabular
data_table.enable_dataframe_formatter()

In [ ]:
# Listando conjuntos de dados na bucket
blobList = [x.name for x in bucket.list_blobs(prefix = prefix_name)]

# dataframe lista blobs
blobDf = pd.DataFrame(blobList)

blobDf

,0
0,dados/tratados/
1,dados/tratados/Veiculos_eletricos_brasil_anfav...
2,dados/tratados/Veiculos_eletricos_brasil_anfav...
3,dados/tratados/dataset_tratados_xlsx/
4,dados/tratados/dataset_tratados_xlsx/2_1_cap_i...
5,dados/tratados/dataset_tratados_xlsx/2_2_cap_i...
6,dados/tratados/dataset_tratados_xlsx/2_2_cap_i...
7,dados/tratados/dataset_tratados_xlsx/2_3_gerac...
8,dados/tratados/fator_capacidade_mwh_2015_2023.csv
9,dados/tratados/iea_ponto_recarga_tratado.csv


In [ ]:
# Desabilita formatação tabular
data_table.disable_dataframe_formatter()

In [ ]:
# Renomeando coluna
blobDf.columns = ['path']

# Lista com resultados
blobPath = blobDf['path'].tolist()

#### Seleção de dados

In [ ]:
# Seleção por index conforme caminho da tabela acima
index = int(input('Infome o número do índice correspondente ao arquivo, conforme tabela acima:' ))

# Seleção do gsutil do arquivo
path = f'gs://{bucket_name}/{blobPath[index]}'

archive= blobPath[index][len(blobPath[0]):]
outpath = f'gs://{bucket_name}/dados/backup/{archive}'

print(f'>>> arquivo : {archive}\n>>> gsutil: {path}\n>>> saida: {outpath[:-4]}.parquet')

Infome o número do índice correspondente ao arquivo, conforme tabela acima:12
>>> arquivo : vendas_veiculoseletricosBrasil_2012_2023.csv
>>> gsutil: gs://projeto-final-ad2-e8/dados/tratados/vendas_veiculoseletricosBrasil_2012_2023.csv
>>> saida: gs://projeto-final-ad2-e8/dados/backup/vendas_veiculoseletricosBrasil_2012_2023.parquet


In [ ]:
# Configuração do blob
blob = bucket.blob(blobPath[index])

# Download do arquivo
blob.download_to_filename(f'/content/{archive}')

In [ ]:
downloadFile = f'/content/{archive}'

# Read

In [ ]:
# Leitura do arquivo da GCS
#df = pd.read_csv(path, skiprows = [random.randint(0, 2000000)],nrows = 200000)
df = pd.read_csv(path)

Tratamento de erros

In [ ]:
#try:
#    df['CEP'] = pd.to_numeric(df['CEP'], errors='coerce')  # Convertendo 'CEP' para numérico
#except pd.errors.ParserError:
#    print("Erro ao ler o CSV. Verifique o formato dos dados.")

# Load

In [ ]:
# Carregamento arquivo no GCS
df.to_parquet(outpath, index=False)